Une table appelée population, contenant la population de chaque pays pour chaque année comprise entre 2012 et la dernière année disponible à la date où vous aurez consulté le site de la FAO. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.


Question 15 : Proposez une clé primaire pertinente pour cette table 'code_pays' 'année'

In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
q15 = pd.read_csv("pop20122013.csv")
population = q15.loc[:,['Pays','Code Pays','Année','Valeur']]
population.columns = ["pays","code_pays","année","Population"]
population['Population'] = ((population['Population'])*1000).astype(int)
population = population[population["pays"] != "Chine"]
population = population.rename(columns={"Population":"population"})
population = population.set_index(['code_pays', 'année'])#clé primaire
#population.to_csv("population.csv", index = True)
population.head(10)

pays  population
code_pays année                            
2         2012      Afghanistan    29825000
          2013      Afghanistan    30552000
202       2012   Afrique du Sud    52386000
          2013   Afrique du Sud    52776000
3         2012          Albanie     3162000
          2013          Albanie     3173000
4         2012          Algérie    38482000
          2013          Algérie    39208000
79        2012        Allemagne    82800000
          2013        Allemagne    82727000

Une table appelée dispo_alim contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les informations suivantes:
la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
disponibilité alimentaire en tonnes
disponibilité alimentaire en Kcal/personne/jour
disponibilité alimentaire de protéines en g/personne/jour
disponibilité alimentaire de matières grasses en g/personne/jour
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .


Question 16 : Proposez une clé primaire pertinente pour cette table."code_pays" "année" "code_produit"

In [36]:
# Import des données des bilans alimentaires
veg = pd.read_csv("q16veg.csv")
ani = pd.read_csv("q16ani.csv")

# Ajout de la variable origin
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

temp = temp[temp["Pays"] != "Chine"]
temp.columns = ['Code Domaine', 'Domaine', 'code_pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code Année', 'année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'origin']
dispo_alim = temp.pivot_table(
    index=["code_pays","Pays","Code Produit","Produit","année","origin"],
    columns = ["Élément"], values=["Valeur"], aggfunc=sum)
del temp
dispo_alim = dispo_alim.reset_index()
dispo_alim = dispo_alim.fillna(0)
dispo_alim = dispo_alim.set_index(["code_pays", "année"])
dispo_alim = pd.merge(dispo_alim, population,on=["code_pays","année"])
dispo_alim.columns = ["pays","code_produit","produit","origin","dispo_alim_kcal_p_j","Disponibilité alimentaire en quantité (kg/personne/an)","dispo_mat_gr","dispo_prot","xxx","Population"]
dispo_alim['dispo_alim_tonnes'] = ((dispo_alim['Disponibilité alimentaire en quantité (kg/personne/an)'])*(dispo_alim['Population']))/1000
dispo_alim = dispo_alim.loc[:,['pays','produit','code_produit','origin','dispo_alim_tonnes','dispo_alim_kcal_p_j','dispo_prot','dispo_mat_gr']]
dispo_alim = dispo_alim.reset_index()
dispo_alim = dispo_alim.set_index(["code_pays", "année", "code_produit"])#clé primaire
#dispo_alim.to_csv("dispo_alim.csv", index = True)
dispo_alim.head(5)

pays produit   origin  dispo_alim_tonnes  \
code_pays année code_produit                                                
1         2012  2511          Arménie     Blé  vegetal          376884.86   
                2513          Arménie    Orge  vegetal             504.73   
                2514          Arménie    Maïs  vegetal              89.07   
                2515          Arménie  Seigle  vegetal             445.35   
                2516          Arménie  Avoine  vegetal             950.08   

                              dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  
code_pays année code_produit                                                 
1         2012  2511                        995.0       29.69          3.53  
                2513                          1.0        0.03          0.01  
                2514                          0.0        0.01          0.00  
                2515                          1.0        0.03          0.01  
                2516                          2.0        0.08          0.03

Une table appelée equilibre_prod contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les quantités suivantes :
disponibilité intérieure
aliments pour animaux
semences
pertes
transformés
nourriture
autres utilisations
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.


Question 17 : Proposez une clé primaire pertinente pour cette table."code_pays" "année" "code_produit"

In [37]:
q17 = pd.read_csv("q17.csv")
temp = q17[q17["Pays"] != "Chine"]
temp.columns = ['Code Domaine', 'Domaine', 'code_pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code Année', 'année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole']
equilibre_prod = temp.pivot_table(
    index=["code_pays","Pays","Code Produit","Produit","année"],
    columns = ["Élément"], values=["Valeur"], aggfunc=sum)
del temp
equilibre_prod.columns = ["alim_ani","autres_utilisations","dispo_int","nourriture","pertes","semences","transfo"]
equilibre_prod = equilibre_prod.reset_index()
equilibre_prod = equilibre_prod.fillna(0)
equilibre_prod = equilibre_prod.rename(columns={"Pays":"pays"})
equilibre_prod = equilibre_prod.rename(columns={"Code Produit":"code_produit"})
equilibre_prod = equilibre_prod.rename(columns={"Produit":"produit"})
equilibre_prod = equilibre_prod.set_index(["code_pays", "année", "code_produit"])#clé primaire
#equilibre_prod.to_csv("equilibre_prod.csv", index = True)
equilibre_prod.head()

pays produit  alim_ani  autres_utilisations  \
code_pays année code_produit                                                   
1         2012  2511          Arménie     Blé      92.0                  0.0   
          2013  2511          Arménie     Blé      93.0                  0.0   
          2012  2513          Arménie    Orge     121.0                 16.0   
          2013  2513          Arménie    Orge     137.0                 26.0   
          2012  2514          Arménie    Maïs      53.0                  0.0   

                              dispo_int  nourriture  pertes  semences  transfo  
code_pays année code_produit                                                    
1         2012  2511              532.0       377.0    29.0      23.0     10.0  
          2013  2511              554.0       389.0    32.0      30.0     10.0  
          2012  2513              172.0         1.0    14.0      14.0      7.0  
          2013  2513              198.0         0.0    15.0      14.0      7.0  
          2012  2514               58.0         0.0     5.0       0.0      0.0

Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays et pour chaque année comprise entre 2012 et la dernière année disponible. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.


Question 18 : Vous vous en doutez... proposez encore une fois une clé primaire pertinente pour cette table ! 'code_pays' 'année'

In [38]:
q18 = pd.read_csv("q18.csv")
temp = q18[q18["Zone"] != "Chine"]
temp = temp.loc[:,['Zone','Code zone','Année','Valeur']]
temp.columns = ['pays', 'code_pays', 'année', 'nb_personnes']
temp = temp[temp["nb_personnes"] != "<0.1"]
temp = temp.dropna()
temp['nb_personnes'] = (temp['nb_personnes']).astype(float)
temp['nb_personnes'] = ((temp['nb_personnes'])*1000000).astype(int)
temp['année'] = temp['année'].replace("2011-2013", "2012")
temp['année'] = temp['année'].replace("2012-2014", "2013")
temp['année'] = temp['année'].replace("2013-2015", "2014")
temp['année'] = temp['année'].replace("2014-2016", "2015")
temp['année'] = temp['année'].replace("2015-2017", "2016")
temp['année'] = temp['année'].replace("2016-2018", "2017")
sous_nutrition = temp
del temp
sous_nutrition = sous_nutrition.set_index(['code_pays', 'année'])#clé primaire
#sous_nutrition.to_csv("sous_nutrition.csv", index = True)
sous_nutrition.head(15)

pays  nb_personnes
code_pays année                              
2         2012      Afghanistan       7100000
          2013      Afghanistan       7900000
          2014      Afghanistan       8800000
          2015      Afghanistan       9600000
          2016      Afghanistan      10200000
          2017      Afghanistan      10600000
202       2012   Afrique du Sud       2400000
          2013   Afrique du Sud       2600000
          2014   Afrique du Sud       2800000
          2015   Afrique du Sud       3200000
          2016   Afrique du Sud       3400000
          2017   Afrique du Sud       3500000
3         2012          Albanie        200000
          2013          Albanie        200000
          2014          Albanie        200000

Question 19 : Écrivez les requêtes SQL permettant de connaître…

Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

Pour chaque année disponible, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.

Le nombre de lignes de la table renvoyée sera donc égal à 10 fois le nombre d'années disponibles.

La quantité totale (en kg) de produits perdus par pays et par année.La table renvoyée contiendra donc une ligne par couple (pays, année).

Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [39]:
import pandas as pd
import numpy as np
import sqlite3 as sql
database="Projet_3.db"
connection = sql.connect(database)
query = '''SELECT * FROM sous_nutrition'''
df = pd.read_sql_query(query,connection)
df.head()
#test

,code_pays,année,pays,nb_personnes
0,2,2012,Afghanistan,7100000
1,2,2013,Afghanistan,7900000
2,2,2014,Afghanistan,8800000
3,2,2015,Afghanistan,9600000
4,2,2016,Afghanistan,10200000


Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

In [40]:
query = '''SELECT pays,SUM(dispo_prot)/1000*365 AS dispo_prot_kg
FROM dispo_alim 
WHERE année="2013"
GROUP BY pays
ORDER BY(dispo_prot_kg)
DESC LIMIT 10'''
df = pd.read_sql_query(query,connection)
df

,pays,dispo_prot_kg
0,Islande,48.56690
1,Chine - RAS de Hong-Kong,47.11055
2,Israël,46.72000
3,Lituanie,45.39140
4,Maldives,44.64680
5,Finlande,42.90940
6,Luxembourg,41.47860
7,Monténégro,40.84350
8,Pays-Bas,40.68290
9,Albanie,40.65005


In [41]:
query = '''SELECT pays,SUM(dispo_alim_kcal_p_j)*365 AS dispo_kcal
FROM dispo_alim 
WHERE année="2013"
GROUP BY pays
ORDER BY(dispo_kcal) DESC LIMIT 10'''
df = pd.read_sql_query(query,connection)
df

,pays,dispo_kcal
0,Autriche,1376050.0
1,Belgique,1364005.0
2,Turquie,1353420.0
3,États-Unis d'Amérique,1343930.0
4,Israël,1317650.0
5,Irlande,1314730.0
6,Italie,1305970.0
7,Luxembourg,1292100.0
8,Égypte,1284070.0
9,Allemagne,1278595.0


Pour chaque année disponible, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant. Le nombre de lignes de la table renvoyée sera donc égal à 10 fois le nombre d'années disponibles.

In [42]:
query = '''SELECT  *
FROM    (SELECT pays,année, SUM(dispo_prot)/1000*365 AS dispo_prot_kg
FROM dispo_alim
WHERE année="2013"
GROUP BY pays,année
ORDER BY dispo_prot_kg
ASC LIMIT 10)
UNION
SELECT  *
FROM    (SELECT pays,année, SUM(dispo_prot)/1000*365 AS dispo_prot_kg
FROM dispo_alim
WHERE année="2012"
GROUP BY pays,année
ORDER BY dispo_prot_kg
ASC LIMIT 10)
ORDER BY année,dispo_prot_kg
'''
df = pd.read_sql_query(query,connection)
df

,pays,année,dispo_prot_kg
0,Libéria,2012,13.89555
1,Guinée-Bissau,2012,15.98700
2,Mozambique,2012,17.12580
3,Haïti,2012,17.23165
4,Madagascar,2012,17.24625
5,Congo,2012,18.06385
6,Zimbabwe,2012,18.51280
7,République centrafricaine,2012,19.27200
8,Sao Tomé-et-Principe,2012,19.35960
9,Ouganda,2012,19.59685


La quantité totale (en kg) de produits perdus par pays et par année. La table renvoyée contiendra donc une ligne par couple (pays, année).

In [43]:
query = '''SELECT pays,année, SUM(pertes)*1000000 AS pertes_kg
FROM equilibre_prod
GROUP BY pays,année
ORDER BY pertes_kg
DESC
'''
df = pd.read_sql_query(query,connection)
df

,pays,année,pertes_kg
0,"Chine, continentale",2013,8.957500e+10
1,"Chine, continentale",2012,8.784900e+10
2,Brésil,2013,7.591400e+10
3,Brésil,2012,7.389700e+10
4,Inde,2013,5.593000e+10
5,Inde,2012,5.164000e+10
6,Nigéria,2013,1.985400e+10
7,Nigéria,2012,1.965000e+10
8,Indonésie,2012,1.336900e+10
9,Indonésie,2013,1.308100e+10


Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

In [44]:
query = '''SELECT population.pays,population.année,sous_nutrition.nb_personnes,population.population,
CAST(nb_personnes AS float) / CAST(population AS float) AS ratio
FROM population
LEFT JOIN sous_nutrition
ON population.pays = sous_nutrition.pays
AND population.année = sous_nutrition.année
WHERE population.année = '2013'
ORDER BY ratio DESC
LIMIT 10
'''
df = pd.read_sql_query(query,connection)
df

,pays,année,nb_personnes,population,ratio
0,Haïti,2013,5200000,10317000,0.504022
1,Zambie,2013,7000000,14539000,0.481464
2,Zimbabwe,2013,6600000,14150000,0.466431
3,République centrafricaine,2013,2000000,4616000,0.433276
4,République populaire démocratique de Corée,2013,10600000,24895000,0.425788
5,Congo,2013,1800000,4448000,0.404676
6,Tchad,2013,4900000,12825000,0.382066
7,Angola,2013,8100000,21472000,0.377235
8,Libéria,2013,1600000,4294000,0.372613
9,Madagascar,2013,8199999,22925000,0.357688


Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [45]:
query = '''SELECT produit,SUM(autres_utilisations)/SUM(dispo_int) AS ratio
FROM equilibre_prod
WHERE année="2013"
GROUP BY produit
ORDER BY ratio
DESC LIMIT 10'''
df = pd.read_sql_query(query,connection)
df

,produit,ratio
0,"Alcool, non Comestible",1.001288
1,Huil Plantes Oleif Autr,0.754744
2,Huile de Palmistes,0.703957
3,Huile de Palme,0.698109
4,Girofles,0.646154
5,Huile de Colza&Moutarde,0.551290
6,Graisses Animales Crue,0.470219
7,Huiles de Poissons,0.448068
8,Huile de Soja,0.416172
9,Plantes Aquatiques,0.379299


Question 20 : pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)

Alcool, non Comestible:
-pharmacie : antiseptique(a préparation de la peau avant injection ou ponction ;
l'antisepsie des petites plaies peu étendues, en badigeon avec une compresse ou en pansement humide), désinfectant (petits objets;instruments médicaux, ciseaux, pinces à épiler, thermomètres..et des surfaces inertes)
-Sous forme d'alcool à brûler il sert à alimenter les réchauds à alcool notamment (utilisation pour les réchaud à fondue). Il est constitué d'un mélange d'alcool éthylique (pour 90 %) et de méthanol (de 5 à 10 %) servant de dénaturant.
-L'alcool ménager est de l'éthanol quasi pur à 95 %, dénaturé pour ne pas être bu, et destiné à l'entretien ménager. Les industriels peuvent le parfumer pour en rendre l'utilisation agréable.
https://medicament.ooreka.fr/astuce/voir/724967/alcool-a-70
https://fr.wikipedia.org/wiki/Alcool_d%C3%A9natur%C3%A9

Huil Plantes Oleif Autr:
Code Produit: 2586
Produit Huil: Plantes Oleif Autr
Description :Composition par défaut: 264 Beurre de karité, 266 Huile de ricin, 276 Huile d'abrasin, 278 Huile de jojoba, 281 Huile de carthame, 297 Huile d'oeillette, 306 Suif végétal, 307 Huile de stillingia, 313 Huile de kapok, 334 Huile de lin, 337 Huile de chènevis, 340 Huile végétale nda, 664 Cacao, beurre, 1241 Margarine liquide, 1242 Margarine et graisses alimentaire compactes, 1273 Huile de ricin hydrogénée, 1274 Huile bouillie, déshydraté, etc., 1275 Huile et graisse végétale hydrogénée
utilisations cosmetiques et alimentaires

Une plante oléifère, ou oléagineuse, est une plante dont l'on extrait de l'huile végétale des fruits ou des graines. Par exemple, le colza, le tournesol et l'olivier sont des plantes oléifères. L'huile végétale récupérée de ces plantes peut être destinée à l'alimentation humaine ou à la production de biocarburant pour moteur diesel, sous forme pure ou transestérifiée (EMHV, EEHV)

Huile de Palmistes:
Les débouchés de l’huile de palmiste sont nombreux : savonnerie et cosmétique, oléochimie.
https://www.cirad.fr/nos-recherches/filieres-tropicales/palmier-a-huile/plante-et-usages

Huile de Palme:
Elle entre dans la fabrication de dérivés à usages industriels : acides gras, savons et cosmétiques, savons industriels, encres, résines, esters méthyliques.
L’huile de palme peut aussi être utilisée comme carburant dans les moteurs diesels, soit à partir de l’huile pure, soit après transformation en esther méthylique, mélangé au gazole.
https://www.cirad.fr/nos-recherches/filieres-tropicales/palmier-a-huile/plante-et-usages


Girofles:
Les propriétés antiseptiques et anesthésiques de ces boutons floraux sont reconnues depuis très longtemps et proposées dans les douleurs dentaires. Il entre dans la composition du khôl, primitivement onguent ophtalmique.
En cuisine, il est présent dans le pain d'épices, les biscuits en mélange avec la cannelle, le pot-au-feu, les marinades, la choucroute et il est indispensable à la plupart des currys. Aux Pays Bas, le clou de girofle entier est utilisé pour parfumer un vieux fromage maigre de Frise à pâte dure appelée Nagelkaas (fromage à clous) ou Kanterkaas (AOP)4,5. En Afrique du Nord, le clou de girofle est utilisé en infusion avec le thé.
Le clou de girofle sert de parfum d'ambiance sous forme d'une version végétale de la pomme d'ambre que l'on fabrique en piquant toute la surface d'une orange de clous de girofle.
95 % de la production mondiale de clous de girofle est utilisé pour la fabrication des kreteks, cigarettes indonésiennes.
https://fr.wikipedia.org/wiki/Giroflier